### Example: Using Enola for Dynamically Field-Programmable Qubit Arrays Based on Neutral Atoms

This notebook demonstrates how to use our compiler, Enola, to compile and execute quantum programs on a single-zone neutral atom architecture. The architecture is based on dynamically field-programmable qubit arrays, where qubits can be reconfigured during circuit execution.

***Prerequisite***
Download Enola from [Github](https://github.com/UCLA-VAST/Enola)

In [ ]:
!git clone https://github.com/UCLA-VAST/Enola.git
!pip3 install networkx
!pip3 install matplotlib
!pip3 install 'qiskit>=1'

***Import***

In [1]:
from enola.enola import Enola
import qiskit # for generic circuits with dependency. Convert the gates in circuits into hardware-supported gate set

### Compile a Circuit

***Step 1: Compiler intialization***

- Default compiler: Use edge coloring algorithm for commutable gate scheduling
    - Placement optimization level 2: simulated annealing with dynamic placement
    - Routing optimization level 1: maximal independent set with sorting heuristic for routing strategy, no restriction for vectex number in solving MIS.
    - For animation code generation, set `full_code=True`

- Scalable compiler: 
    - Placement optimization level 0: Use trivial layout, return to initial mapping after each Rydberg stage 
    - Routing optimization level 0: using maximal independent set for routing  and limit the number of vertices to 1000 in solving MIS

In [2]:
commutable_circuit_name = 'qaoa'
default_compiler = Enola(
        name=commutable_circuit_name,
        dir='./results/',
        full_code=True)

scalable_compiler = Enola(
        name=commutable_circuit_name,
        dir='./results/',
        placement_opt=0,
        routing_opt=0,
        full_code=True)

***Step 2: Set Architecture***

Set architecture dimension by the method `setArchitecture(bounds: Sequence[int])`, where `bounds = [slm_num_row, slm_num_col, aod_num_row, aod_num_col]` specifies the number of columns and rows in the SLM and AOD arrays. Note that 1. the SLM site number, i.e., `slm_num_row`*`slm_num_col`, should be greater than or equal to the number of program qubits in the circuit, and 2. the size of AOD array should not be smaller than the size of SLM arrays.

In [3]:
# an example of setting an architecture with an SLM of 10 columns and rows, and an AOD with 10 columns and rows
default_compiler.setArchitecture([10, 10, 10, 10])

***Step 3: Set Program***

Set quantum program by the method `setProgram(program: Sequence[Sequence[int]])`, where `program` is a list of two-qubit gates in the circuit.

In [4]:
# an example of setting the QAOA circuit
# circuits with all commutable gates
graph_edge_list = [[0, 1], [0, 2], [0, 4], [1, 3], [1, 5], [2, 4], [2, 3], 
                   [3, 5], [4, 5]]

default_compiler.setProgram(graph_edge_list)

***Step 4: Compile the Circuit***

Compile the circuit by method `solve()`. 
- `save_file` specifies whether to save 1. the output program list and 2. the detailed code (if `full_code=True` when initializing the compiler) for animation generation to json files in the folder `results/code`, and 3. the runtime analysis to a json file to the folder `results/time`.

In [5]:
program_list_qaoa = default_compiler.solve(save_file=True)

[INFO] Enola: Start Solving
[INFO] Enola: Run scheduling
[INFO] Enola: Time for scheduling: 0.00031113624572753906s
[INFO] Enola: Start SA-based placement
[INFO] Enola: SA-Based Placer: Iter 0, cost: 8.900000
[INFO] Enola: Time for placement: 18.168779134750366s
[INFO] Enola: Finding a mapping for Rydberg stage 2/4
[INFO] Enola: Start SA-based placement
[INFO] Enola: SA-Based Placer: Iter 0, cost: 8.900000
[INFO] Enola: Solve for Rydberg stage 2/4.
[INFO] Enola: Finding a mapping for Rydberg stage 3/4
[INFO] Enola: Start SA-based placement
[INFO] Enola: SA-Based Placer: Iter 0, cost: 7.800000
[INFO] Enola: Solve for Rydberg stage 3/4.
[INFO] Enola: Finding a mapping for Rydberg stage 4/4
[INFO] Enola: Start SA-based placement
[INFO] Enola: SA-Based Placer: Iter 0, cost: 6.500000
[INFO] Enola: Solve for Rydberg stage 4/4.
[INFO] Enola: Solve for Rydberg stage 5/4.
[INFO] Enola: Time for routing: 0.0006000995635986328s
[INFO] Enola: Toal Time: 66.8243899345398s


#### The other example for compiling a generic circuit

Initialize a compiler for generic circuits with gate dependency by setting `dependency=True`.

In [8]:
generic_circuit_name = "qasm_exp/ising_n26.qasm"
# generic circuits with dependencies
# extract two-qubit gates from the circuit
list_gate_two_qubit = []
with open(generic_circuit_name, 'r') as f:
    qasm_str = f.read()
    circuit = qiskit.QuantumCircuit.from_qasm_str(qasm_str)
    cz_circuit = qiskit.transpile(circuit, basis_gates=["cz", "id", "u2", "u1", "u3"])
    instruction = cz_circuit.data
    for ins in instruction:
        if ins.operation.num_qubits == 2:
            list_gate_two_qubit.append((ins.qubits[0]._index, ins.qubits[1]._index))
print("list_gate_two_qubit: ", list_gate_two_qubit)

# use the scalable compiler with dependency = True
scalable_generic_compiler = Enola(
        name="ising_n26",
        dir='./results/',
        trivial_layout = True,
        dependency=True,
        routing_strategy="maximalis",
        reverse_to_initial=True,
        use_window=True)

# set architecture
scalable_generic_compiler.setArchitecture([10, 10, 10, 10])

# set program
scalable_generic_compiler.setProgram(list_gate_two_qubit)

# solve the problem
program_list_generic = scalable_generic_compiler.solve(save_file=True)

list_gate_two_qubit:  [(0, 1), (0, 1), (2, 3), (2, 3), (1, 2), (1, 2), (4, 5), (4, 5), (3, 4), (3, 4), (6, 7), (6, 7), (5, 6), (5, 6), (8, 9), (8, 9), (7, 8), (7, 8), (10, 11), (10, 11), (9, 10), (9, 10), (12, 13), (12, 13), (11, 12), (11, 12), (14, 15), (14, 15), (13, 14), (13, 14), (16, 17), (16, 17), (15, 16), (15, 16), (18, 19), (18, 19), (17, 18), (17, 18), (20, 21), (20, 21), (19, 20), (19, 20), (22, 23), (22, 23), (21, 22), (21, 22), (24, 25), (24, 25), (23, 24), (23, 24)]
[INFO] Enola: Start Solving
[INFO] Enola: Run scheduling
[INFO] Enola: Time for scheduling: 2.6941299438476562e-05s
10
[INFO] Enola: Time for placement: 6.9141387939453125e-06s
[INFO] Enola: Solve for Rydberg stage 2/4.
[INFO] Enola: Solve for Rydberg stage 3/4.
[INFO] Enola: Solve for Rydberg stage 4/4.
[INFO] Enola: Solve for Rydberg stage 5/4.
[INFO] Enola: Time for routing: 0.0014810562133789062s
[INFO] Enola: Toal Time: 0.006429910659790039s


### Circuit Fidelity Simulation
$f=(f_{1})^{g_1} \cdot \underbrace{(f_{2})^{g_2} \cdot (f_\text{exc})^{|Q| S - 2g_2}}_\text{two-qubit gate} \cdot \underbrace{(f_\text{trans})^{N_\text{trans}}}_\text{atom transfer} \cdot \underbrace{\Pi_{q\in Q}\ (1-T_q/T_2)}_\text{decoherence}$
- $f_{1}$: Single-qubit gate fidelity 
- $f_{2}$: Two-qubit gate fidelity 
- $f_\text{exc}$: Idle qubit excitation fidelity 
- $f_\text{trans}$: Atom transfer fidelity 
- $g_1$: Number of single-qubit gates
- $g_2$: Number of two-qubit gates
- $Q$: Set of program qubits
- $S$: Number of stage
- $N_\text{trans}$: Number of atom transfers
- $T_q$: Idle time for qubit $q$

***Run simulator***: 
Run `python3 simulator.py <P> --arch_param <A> --fidelity_param <F>` where `<P>` is the filename to the program instruction lists, `<A>` is the json file constaining architecture parameters, `<F>` is the json file containing fidelity parameters.

Default: 
- `fidelity_param` is "hardware_spec/compute_store_arch_fidelity.json"
    - R_B: 6um
    - AOD_SEP: 2um
    - RYD_SEP: 15um
    - SITE_SLMS: 2um
    - SLM_SEP: 2um
    - SITE_WIDTH: 4um
    - T_RYDBERG: 0.36us
    - T_ACTIVATE: 15us

- `arch_param` is "hardware_spec/compute_store_arch.json"
    - 2QG: 0.995
    - 1QG: 0.9997
    - AT: 0.999
    - T: 1.5e6us

The program will save the json file consisting of fidelity anaylsis to the folder `results/fidelity/`

In [11]:
# example command
!python3 simulator.py results/code/qaoa_code.json

### Animation Generation

***Prerequisite*** 

Install [ffmpeg](https://www.ffmpeg.org)

To generate animation, initialize `Animator` with the full code file. For graph-induced circuit, we can show a graph located next to the architecture to indicate the circuit execution condition by setting `show_graph=True` and specifying `edges`.



In [10]:
# import animator
from animation import Animator

# generate animation for qaoa circuit
Animator(
        "results/code/qaoa_code_full.json",
        show_graph=True,
        edges=graph_edge_list,
        dir='./results/animations/'
    )

# generate animation for ising circuit
Animator(
        "results/code/ising_n26_code_full.json",
        show_graph=False,
        edges=[],
        dir='./results/animations/'
    )